In [2]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

# HAPPINESS

## STEP 1: IMPORT

In [3]:
# IMPORTING orignal input csvs:
df_2019 = pd.read_csv('happiness_2019.csv')
df_2018 = pd.read_csv('happiness_2018.csv')
df_2017 = pd.read_csv('happiness_2017.csv')
df_2016 = pd.read_csv('happiness_2016.csv')
df_2015 = pd.read_csv('happiness_2015.csv')

# Copy dfs to w(=working)dfs:   
wdf19 = df_2019.copy()
wdf18 = df_2018.copy()
wdf17 = df_2017.copy()
wdf16 = df_2016.copy()
wdf15 = df_2015.copy()

# STEP 2: COLUMNS SELECTION

#### Select columns

In [4]:
# first review heads of all dfs 
wdf15.head(1)

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738


In [5]:
wdf16.head(1)

,Country,Region,Happiness Rank,Happiness Score,Lower Confidence Interval,Upper Confidence Interval,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Denmark,Western Europe,1,7.526,7.46,7.592,1.44178,1.16374,0.79504,0.57941,0.44453,0.36171,2.73939


In [6]:
wdf17.head(1)

,Country,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027


In [7]:
wdf18.head(1)

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.632,1.305,1.592,0.874,0.681,0.202,0.393


In [8]:
wdf19.head(1)

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.769,1.34,1.587,0.986,0.596,0.153,0.393


#### DECISION: 
       We want to keep only those countries, that are in all five tables and have values for score and rank everywhere

In [9]:
# reviws shows: tables have different column names, so we rename them with names including the year
# get the selected columns, drop na vals, rename and udpate them with countries as index:

In [10]:
wdf19 = wdf19.get(['Country or region','Score','Overall rank'])
wdf19 = wdf19.dropna().rename(columns={'Country or region':'Country','Score':'Score_19','Overall rank':'Rank_19'})

In [11]:
wdf18 = wdf18.get(['Country or region','Score','Overall rank'])
wdf18 = wdf18.dropna().rename(columns={'Country or region':'Country','Score':'Score_18','Overall rank':'Rank_18'})

In [12]:
wdf17 = wdf17.get(['Country', 'Happiness.Rank', 'Happiness.Score']) 
wdf17 = wdf17.dropna().rename(columns={'Country':'Country', 'Happiness.Rank':'Rank_17', 'Happiness.Score':'Score_17'})

In [13]:
wdf16 = wdf16.get(['Country','Happiness Rank', 'Happiness Score'])
wdf16 = wdf16.dropna().rename(columns={'Country':'Country','Happiness Rank':'Rank_16', 'Happiness Score':'Score_16'})

In [14]:
wdf15 = wdf15.get(['Country','Happiness Rank', 'Happiness Score'])
wdf15 = wdf15.dropna().rename(columns={'Country':'Country','Happiness Rank':'Rank_15', 'Happiness Score':'Score_15'})

### Filter 1: Countries included in all five tables

In [15]:
# concat country col of all wdfs an save them in temp df

temp = pd.concat([wdf15, wdf16,wdf17,wdf18,wdf19], axis=0)['Country']
temp = pd.DataFrame(temp)

# see len before:
len(temp)

782

In [16]:
# only keep counts of five (=> countries in all five tables)

temp = pd.DataFrame(temp[temp['Country'].map(temp['Country'].value_counts())==5])

# see len after:
len(temp)

705

In [17]:
# get uniques and save to countrydf and countrylist

uni = list(temp['Country'].unique())
d = {'Country':uni}
countrydf = pd.DataFrame(d).sort_values('Country')
countrylist = countrydf['Country'].to_list()

In [18]:
# set country to index and update wdfs

wdf15 = wdf15.pivot_table(index='Country')
wdf16 = wdf16.pivot_table(index='Country')
wdf17 = wdf17.pivot_table(index='Country')
wdf18 = wdf18.pivot_table(index='Country')
wdf19 = wdf19.pivot_table(index='Country')

In [19]:
# function to drop countries and update to n(ew)df

def dropcounty(df):
    indexlist = df.index.to_list()
    removerlist = []
    for x in indexlist:
        if x not in countrylist:
            removerlist.append(x)
    ndf = df.copy()
    ndf = ndf.drop(x for x in removerlist)
    return ndf

In [20]:
# update all wdfs to ndfs

ndf15 = dropcounty(wdf15)
ndf16 = dropcounty(wdf16)
ndf17 = dropcounty(wdf17)
ndf18 = dropcounty(wdf18)
ndf19 = dropcounty(wdf19)

# STEP 3: JOIN SELECTION

In [21]:
# new res(ult)df with all scores and ranks

resdf = ndf15.join(ndf16).join(ndf17).join(ndf18).join(ndf19)

# confirm
resdf.head()

,Rank_15,Score_15,Rank_16,Score_16,Rank_17,Score_17,Rank_18,Score_18,Rank_19,Score_19
Country,,,,,,,,,,
Afghanistan,153,3.575,154,3.360,141,3.794,145,3.632,154,3.203
Albania,95,4.959,109,4.655,109,4.644,112,4.586,107,4.719
Algeria,68,5.605,38,6.355,53,5.872,84,5.295,88,5.211
Argentina,30,6.574,26,6.650,24,6.599,29,6.388,47,6.086
Armenia,127,4.350,121,4.360,121,4.376,129,4.321,116,4.559


### Filter 2 & 3: Countries with values for rank and score for all five years

In [22]:
# confirm: zero na-vals
resdf.isna().sum()
    # values for scores and ranks are ok

Rank_15     0
Score_15    0
Rank_16     0
Score_16    0
Rank_17     0
Score_17    0
Rank_18     0
Score_18    0
Rank_19     0
Score_19    0
dtype: int64

# STEP 4: CAPITALS

In order to match these countries with input from weather data, countries need to be exported and imported again by groupmates who run the weather requests. Weather requests are going to be made by capital. In order to match happyness-input and weather-input again in further analysis, capitals exported by groupmates will be imported here again.

In [23]:
# exporting countries 
happyness = resdf.copy()

# happyness.to_csv("happy.csv")
    # -> as comment to not export again

capitaldf = pd.read_csv('capitalsofcountries.csv')
capitaldf

,Unnamed: 0,Country,Capital
0,0,Mauritius,Port Louis
1,1,Austria,Vienna
2,2,Iceland,Reykjavik
3,3,South Korea,Seoul
4,4,Bosnia and Herzegovina,Sarajevo
...,...,...,...
136,136,New Zealand,Wellington
137,137,Yemen,Sana'a
138,138,Hungary,Budapest
139,139,Kyrgyzstan,Bishkek


In [24]:
wdf = capitaldf.copy().get(['Country','Capital']).set_index('Country')

### JOIN Capitals on Happyness

In [25]:
happyfinal = resdf.copy().join(wdf)

In [26]:
happyfinal

,Rank_15,Score_15,Rank_16,Score_16,Rank_17,Score_17,Rank_18,Score_18,Rank_19,Score_19,Capital
Country,,,,,,,,,,,
Afghanistan,153,3.575,154,3.360,141,3.794,145,3.632,154,3.203,Kabul
Albania,95,4.959,109,4.655,109,4.644,112,4.586,107,4.719,Tirana
Algeria,68,5.605,38,6.355,53,5.872,84,5.295,88,5.211,Algiers
Argentina,30,6.574,26,6.650,24,6.599,29,6.388,47,6.086,Buenos Aires
Armenia,127,4.350,121,4.360,121,4.376,129,4.321,116,4.559,Yerevan
...,...,...,...,...,...,...,...,...,...,...,...
Venezuela,23,6.810,44,6.084,82,5.250,102,4.806,108,4.707,Caracas
Vietnam,75,5.360,96,5.061,94,5.074,95,5.103,94,5.175,Hanoi
Yemen,136,4.077,147,3.724,146,3.593,152,3.355,151,3.380,Sana'a


# STEP 5: EXPORT HAPPYNESS

In [27]:
# export cleaned df for further processing

# happyfinal.to_csv("happyclean.csv")
    # comment to not export again